In [28]:
import torch 
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

from API_utils.dataset_FEGS import API_FEGS_Class


In [29]:
DATASET_PATH = "C:/Users/asus/Desktop/API/dataset/one_to_one.xls"
DATASET_MAT_PATH = "C:/Users/asus/Desktop/API/dataset/one_to_one.mat"
TEST_DATASET_PATH = "C:/Users/asus/Desktop/API/dataset/test.xlsx"
TEST_DATASET_MAT_PATH = "C:/Users/asus/Desktop/API/dataset/test.mat"
SAVE_MODEL_PATH = "C:/Users/asus/Desktop/API/Model/"
DEVICE= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCH = 1000
lr=0.0005

In [30]:
print(DEVICE)

cuda:0


In [31]:
train_data = API_FEGS_Class(DATASET_PATH,DATASET_MAT_PATH,'abc')
test_data = API_FEGS_Class(TEST_DATASET_PATH,TEST_DATASET_MAT_PATH,'test')

In [32]:
class NeuralNet(nn.Module):
    def __init__(self, lrate = 0.001, loss_fn = nn.CrossEntropyLoss()):
        super(NeuralNet, self).__init__()


        self.nn = nn.Linear(642,1)

        self.loss_fn = loss_fn
        self.lrate=lrate        
        self.optimizer=optim.SGD(self.parameters(),self.lrate,momentum=0.9)
    def forward(self,x):
        """Performs a forward pass through your neural net (evaluates f(x)).

        @param x: an (N, in_size) Tensor
        @return y: an (N, out_size) Tensor of output from the network
        """
        '''
        extractor for protein 
        '''
        
        x = self.nn(x)
        return x


In [33]:
def HingeLoss(pred, truth):
    # Expect both of shape (N)
    loss_tensor = nn.ReLU()(1-pred*truth)
    return torch.mean(loss_tensor)

In [34]:
train_data_loader = DataLoader(train_data,batch_size=1,shuffle=True)
test_data_loader = DataLoader(test_data,batch_size=1,shuffle=True)
Net = NeuralNet().to(DEVICE)
optimizer = optim.Adam(params=Net.parameters(),lr=lr)
loss_fn = HingeLoss


In [35]:
def evaluate(model_path,test_data_loader,device=DEVICE):
    count=0
    acc = 0
    model = torch.load(model_path)
    for idx, data in enumerate(test_data_loader):
        
        api_input, api_label = data
        if api_input.shape != torch.Size([1, 642]):
            continue
        count+=1
        api_input = api_input.to(DEVICE)
        api_input = api_input.to(dtype=torch.float32)
        api_label = api_label.to(DEVICE)
        # print(rna_input)
        #api_label = torch.unsqueeze(api_label,dim=0)
        output = model(api_input).to(DEVICE)
        if(torch.round(torch.sigmoid(output))==api_label[0]):
            acc+=1

    
    print("Accuracy",acc/count)

In [36]:
for epoch in range(EPOCH):
    for data in tqdm(train_data_loader):
        api_input, api_label = data
        api_input = api_input.to(DEVICE)
        api_label = api_label.to(DEVICE)
        if api_input.shape != torch.Size([1, 642]):
            continue
        # print(api_label)
        api_input = api_input.to(dtype=torch.float32)
        # print(protein_input)
        api_label = torch.unsqueeze(api_label,dim=0).to(dtype=torch.float32)

        output = Net(api_input).to(DEVICE)
        
        # print(output)
        optimizer.zero_grad()    
        Loss = loss_fn(output, api_label)  
        Loss.backward()  
        optimizer.step()
        

    print("Loss",Loss.item())
    save_path = SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}.pth'
    torch.save(Net, save_path)
    evaluate(SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}.pth',test_data_loader)
    
torch.save(Net, SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}_final.pth')
print('CSX')

100%|██████████| 1160/1160 [00:01<00:00, 1010.66it/s]


Loss 0.0
Accuracy 0.2491349480968858


100%|██████████| 1160/1160 [00:01<00:00, 1139.49it/s]


Loss 0.0
Accuracy 0.2491349480968858


100%|██████████| 1160/1160 [00:01<00:00, 1084.11it/s]


...

Loss 1.0
Accuracy 0.2491349480968858


100%|██████████| 1160/1160 [00:01<00:00, 885.31it/s]


Loss 1.0
Accuracy 0.2491349480968858


100%|██████████| 1160/1160 [00:01<00:00, 1125.12it/s]


Loss 0.0


In [ ]:
model=torch.load(SAVE_MODEL_PATH+f'Epoch={epoch}_lr={lr}_final.pth')

In [ ]:

acc=0
count=0
for idx, data in enumerate(test_data_loader):
    api_input, api_label = data
    count+=1
    rna_input = api_input[:,0:64]
    protein_input = api_input[:,64:909]
    rna_input = torch.unsqueeze(rna_input,dim=0)
    protein_input = torch.unsqueeze(protein_input,dim=0)
    #api_label = torch.unsqueeze(api_label,dim=0)
    rna_input = rna_input.to(dtype=torch.float32)
    protein_input = protein_input.to(dtype=torch.float32)
    output = model(rna_input,protein_input)
    if(torch.argmax(output)==api_label[0]):
        acc+=1

    
print(acc/count)

